In [1]:
!pip install -q scikit-learn numpy pandas altair kaggle==1.6.6 graphviz imbalanced-learn
!pip install -q xgboost lightgbm catboost statsmodels plotly optuna tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.8 MB/s eta 0:00:00


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (10.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 124926 files and direc

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#최대한 간결하게 적느라 빼놓았지만,
#.head(), .info(), .value_counts(), .isnull().sum() 등을 활용하여
#전처리 후 데이터가 어떻게 변화되는지 확인하는 것이 좋을 것 같습니다!

In [5]:
#한글 폰트(아래 feature importance 그래프에서 한글이 깨지는 것을 방지하기 위해)
plt.rc('font', family='NanumBarunGothic')

# Data Load

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train = pd.read_csv('/content/drive/My Drive/Aimers6/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/My Drive/Aimers6/test.csv').drop(columns=['ID'])

In [8]:
X = train#.drop('임신 성공 여부', axis=1) #몇몇 피처는 target 인코딩 할 것이기 때문에, 그 후에 드랍하였습니다.
y = train['임신 성공 여부']

# Data Pre-processing

In [9]:
# "알 수 없음"을 "만43-44세"로 변경하였는데, 다른 나이 구간으로 해보셔도 될 것 같습니다.
X['시술 당시 나이'] = X['시술 당시 나이'].replace('알 수 없음', '만43-44세')
test['시술 당시 나이'] = test['시술 당시 나이'].replace('알 수 없음', '만43-44세')

# Target Encoding 적용
age_success_rate = X.groupby("시술 당시 나이")["임신 성공 여부"].mean()
X['시술 당시 나이'] = X['시술 당시 나이'].map(age_success_rate)
test['시술 당시 나이'] = test['시술 당시 나이'].map(age_success_rate)


In [10]:
# 배아 생성 주요 이유 그룹화 (Treatment, Success, Fail)           #EDA 결과 임신 성공률에 차이가 나서 그룹화 해보았음.
reason_mapping = {
    '현재 시술용': 'Treatment',
    '기증용, 배아 저장용, 현재 시술용': 'Success',
    '기증용, 현재 시술용': 'Success',
    '난자 저장용, 현재 시술용': 'Success',
    '배아 저장용, 현재 시술용': 'Success',

    '기증용': 'Fail',
    '기증용, 난자 저장용': 'Fail',
    '기증용, 배아 저장용': 'Fail',
    '난자 저장용, 배아 저장용': 'Fail',
    '난자 저장용': 'Fail',
    '배아 저장용': 'Fail',
    '연구용, 배아 저장용, 연구용': 'Fail',
    '배아 저장용, 연구용, 현재 시술용': 'Fail'
}

# 데이터 변환 적용
X['배아 생성 주요 이유'] = X['배아 생성 주요 이유'].map(reason_mapping).fillna('Treatment')

# 변환 결과 확인
print(X['배아 생성 주요 이유'].value_counts())


배아 생성 주요 이유
Treatment    240025
Fail          12434
Success        3892
Name: count, dtype: int64


In [11]:
# 각 그룹별 평균 임신 성공률 계산
success_rate = X.groupby("배아 생성 주요 이유")["임신 성공 여부"].mean()

# Target Encoding 적용
X['배아 생성 주요 이유'] = X['배아 생성 주요 이유'].map(success_rate)

In [12]:
#test에도 똑같이 적용
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].map(reason_mapping).fillna('Treatment')
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].map(success_rate)

In [13]:
# '난자 출처' 피처의 '알 수 없음' 값을 '본인 제공' 값으로(최빈값으로) 변환
X['난자 출처'] = X['난자 출처'].replace('알 수 없음', '본인 제공')

# 결과 확인
print(X['난자 출처'].value_counts())

난자 출처
본인 제공    240582
기증 제공     15769
Name: count, dtype: int64


In [14]:
test['난자 출처'] = test['난자 출처'].replace('알 수 없음', '본인 제공')

print(test['난자 출처'].value_counts())

난자 출처
본인 제공    84573
기증 제공     5494
Name: count, dtype: int64


In [15]:
X.loc[X["난자 출처"] == "본인 제공", "난자 출처"] = 0
X.loc[X["난자 출처"] == "기증 제공", "난자 출처"] = 1

X.loc[X["정자 출처"] == "배우자 제공", "정자 출처"] = 0
X.loc[X["정자 출처"] == "기증 제공", "정자 출처"] = 1
X.loc[X["정자 출처"] == "미할당", "정자 출처"] = 0
X.loc[X["정자 출처"] == "배우자 및 기증 제공", "정자 출처"] = 0

In [16]:
test.loc[test["난자 출처"] == "본인 제공", "난자 출처"] = 0
test.loc[test["난자 출처"] == "기증 제공", "난자 출처"] = 1

test.loc[test["정자 출처"] == "배우자 제공", "정자 출처"] = 0
test.loc[test["정자 출처"] == "기증 제공", "정자 출처"] = 1
test.loc[test["정자 출처"] == "미할당", "정자 출처"] = 0
test.loc[test["정자 출처"] == "배우자 및 기증 제공", "정자 출처"] = 0

In [17]:
# X = X.drop('불임 원인 - 여성 요인', axis=1)
X = X.drop('난자 채취 경과일', axis=1)
X = X.drop('난자 해동 경과일', axis=1)
X = X.drop('난자 혼합 경과일', axis=1)
# X = X.drop('불임 원인 - 정자 면역학적 요인', axis=1)
# X = X.drop('불임 원인 - 정자 운동성', axis=1)
# X = X.drop('불임 원인 - 자궁경부 문제', axis=1)
# X = X.drop('불임 원인 - 정자 형태', axis=1)
X = X.drop('배아 해동 경과일', axis=1)
# X = X.drop('불임 원인 - 정자 농도', axis=1)
X = X.drop('시술 유형', axis=1)
X = X.drop('임신 시도 또는 마지막 임신 경과 연수', axis=1)
X = X.drop('착상 전 유전 검사 사용 여부', axis=1)
X = X.drop('PGD 시술 여부', axis=1)
X = X.drop('PGS 시술 여부', axis=1)
X = X.drop('배란 유도 유형', axis=1)


In [18]:
# test = test.drop('불임 원인 - 여성 요인', axis=1)
test = test.drop('난자 채취 경과일', axis=1)
test = test.drop('난자 해동 경과일', axis=1)
test = test.drop('난자 혼합 경과일', axis=1)
# test = test.drop('불임 원인 - 정자 면역학적 요인', axis=1)
# test = test.drop('불임 원인 - 정자 운동성', axis=1)
# test = test.drop('불임 원인 - 자궁경부 문제', axis=1)
# test = test.drop('불임 원인 - 정자 형태', axis=1)
test = test.drop('배아 해동 경과일', axis=1)
# test = test.drop('불임 원인 - 정자 농도', axis=1)
test = test.drop('시술 유형', axis=1)
test = test.drop('임신 시도 또는 마지막 임신 경과 연수', axis=1)
test = test.drop('착상 전 유전 검사 사용 여부', axis=1)
test = test.drop('PGD 시술 여부', axis=1)
test = test.drop('PGS 시술 여부', axis=1)
test = test.drop('배란 유도 유형', axis=1)

In [19]:
#'시술 시기 코드'에 One-Hot Encoding 적용
from sklearn.preprocessing import OneHotEncoder

# One-Hot Encoding 적용 열
one_hot_cols = ['시술 시기 코드']

# One-Hot Encoding 적용
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Train 데이터 변환
onehot_encoded = one_hot_encoder.fit_transform(X[one_hot_cols])
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=X.index)

# 기존 데이터와 병합
X = pd.concat([X.drop(columns=one_hot_cols), onehot_encoded_df], axis=1)

In [20]:
#one-hot encoding
test_encoded = one_hot_encoder.transform(test[one_hot_cols])
test_encoded_df = pd.DataFrame(test_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=test.index)

# 기존 데이터와 병합
test = pd.concat([test.drop(columns=one_hot_cols), test_encoded_df], axis=1)

In [21]:
categorical_columns = [
    # "시술 시기 코드", #one-hot
    # "시술 당시 나이", #target-encoder
    # "시술 유형", #삭제
    "특정 시술 유형", #기회되면 target-encoder
    "배란 자극 여부", #ordinal
    # "배란 유도 유형", #삭제
    "단일 배아 이식 여부", #ordinal
    # "착상 전 유전 검사 사용 여부", #삭제
    "착상 전 유전 진단 사용 여부", #ordinal
    "남성 주 불임 원인", #ordinal
    "남성 부 불임 원인", #ordinal
    "여성 주 불임 원인", #ordinal
    "여성 부 불임 원인", #ordinal
    "부부 주 불임 원인", #ordinal
    "부부 부 불임 원인", #ordinal
    "불명확 불임 원인", #ordinal
    "불임 원인 - 난관 질환", #ordinal
    "불임 원인 - 남성 요인", #ordinal
    "불임 원인 - 배란 장애", #ordinal
    "불임 원인 - 여성 요인", #삭제
    "불임 원인 - 자궁경부 문제", #삭제
    "불임 원인 - 자궁내막증", #ordinal
    "불임 원인 - 정자 농도", #삭제
    "불임 원인 - 정자 면역학적 요인", #삭제
    "불임 원인 - 정자 운동성", #삭제
    "불임 원인 - 정자 형태", #삭제
    # "배아 생성 주요 이유", #그룹화, target-encoder
    "총 시술 횟수", #ordinal, 추후에 target 고민
    "클리닉 내 총 시술 횟수", #ordinal, 추후에 target 고민
    "IVF 시술 횟수", #ordinal, 추후에 target 고민
    "DI 시술 횟수", #ordinal
    "총 임신 횟수", #ordinal
    "IVF 임신 횟수", #ordinal
    "DI 임신 횟수", #ordinal
    "총 출산 횟수", #ordinal
    "IVF 출산 횟수", #ordinal
    "DI 출산 횟수", #ordinal
    "난자 출처", #ordinal, '알 수 없음' -> '본인 제공'
    "정자 출처", #ordinal
    "난자 기증자 나이", #ordinal
    "정자 기증자 나이", #ordinal
    "동결 배아 사용 여부", #ordinal, 삭제 고민
    "신선 배아 사용 여부", #ordinal, 삭제 고민
    "기증 배아 사용 여부", #ordinal, 삭제 고민
    "대리모 여부",#ordinal, 삭제 고민
    # "PGD 시술 여부", #삭제
    # "PGS 시술 여부" #삭제
]

In [22]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [23]:
#Ordinal Encoding 적용
from sklearn.preprocessing import  OrdinalEncoder

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [24]:
#수치형 칼럼

numeric_columns = [
    # "임신 시도 또는 마지막 임신 경과 연수", #삭제
    "총 생성 배아 수", #결측치 0 대체
    "미세주입된 난자 수", #결측치 0 대체
    "미세주입에서 생성된 배아 수", #결측치 0 대체
    # "이식된 배아 수", #결측치 2로 대체
    "미세주입 배아 이식 수", #결측치 0 대체
    "저장된 배아 수", #결측치 0 대체
    "미세주입 후 저장된 배아 수", #결측치 0 대체
    "해동된 배아 수", #결측치 0 대체
    "해동 난자 수", #결측치 0 대체
    "수집된 신선 난자 수", #결측치 0 대체
    "저장된 신선 난자 수", #결측치 0 대체
    "혼합된 난자 수", #결측치 0 대체
    "파트너 정자와 혼합된 난자 수", #결측치 0 대체
    "기증자 정자와 혼합된 난자 수", #결측치 0 대체
    # "난자 채취 경과일", #삭제
    # "난자 해동 경과일", #삭제
    # "난자 혼합 경과일", #삭제
    "배아 이식 경과일",
    # "배아 해동 경과일" #삭제
]

In [25]:
#"이식된 배아 수"는 최빈값이 2이므로 따로 빼서 결측치를 2로 대체
numeric_fillna_2 = [
    "이식된 배아 수"
]

In [26]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0) #나머지
X_train_encoded[numeric_fillna_2] = X_train_encoded[numeric_fillna_2].fillna(2) #이식된 배아 수

X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_fillna_2] = X_test_encoded[numeric_fillna_2].fillna(2)

In [27]:
X_train_encoded = X_train_encoded.drop(columns=['임신 성공 여부']) #처음에 안빼줬던 target 변수 빼주기

# Model Training

In [28]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 63 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   시술 당시 나이            256351 non-null  float64
 1   특정 시술 유형            256351 non-null  float64
 2   배란 자극 여부            256351 non-null  float64
 3   단일 배아 이식 여부         256351 non-null  float64
 4   착상 전 유전 진단 사용 여부    256351 non-null  float64
 5   남성 주 불임 원인          256351 non-null  float64
 6   남성 부 불임 원인          256351 non-null  float64
 7   여성 주 불임 원인          256351 non-null  float64
 8   여성 부 불임 원인          256351 non-null  float64
 9   부부 주 불임 원인          256351 non-null  float64
 10  부부 부 불임 원인          256351 non-null  float64
 11  불명확 불임 원인           256351 non-null  float64
 12  불임 원인 - 난관 질환       256351 non-null  float64
 13  불임 원인 - 남성 요인       256351 non-null  float64
 14  불임 원인 - 배란 장애       256351 non-null  float64
 15  불임 원인 - 여성 요인       256351 non-nul

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Model Architecture
model = Sequential([
    Dense(63, activation='relu', input_shape=(63,)),  # Input Layer
    # Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output Layer
])

# Training Parameters
input_size = (None, 63)  # (batch_size, input_features) - batch_size is inferred during training
output_size = (None, 1)  # (batch_size, output_units)
total_dense_layers = 10
regularization = "Batch Normalization, Dropout"
batch_size = 128
optimizer = Adam()
loss = 'binary_crossentropy'
epochs = 50
callbacks = [EarlyStopping(patience=5)]  # Early Stopping callback

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()

# Dummy data for demonstration (replace with your actual data)
import numpy as np
X_train = np.random.rand(1000, 63)  # 1000 samples, 25 features
y_train = np.random.randint(0, 2, size=(1000, 1))  # Binary classification labels

# Train the model
model.fit(X_train_encoded, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_split=0.2)

# Evaluate the model (optional)
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Loss: {loss}, Accuracy: {accuracy}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 63)                  │           4,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,407,809 (5.37 MB)

 Trainable params: 1,406,785 (5.37 MB)

 Non-trainable params: 1,024 (4.00 KB)

Epoch 1/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 79s 47ms/step - accuracy: 0.7408 - loss: 0.5234 - val_accuracy: 0.7425 - val_loss: 0.5023
Epoch 2/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 72s 45ms/step - accuracy: 0.7425 - loss: 0.5026 - val_accuracy: 0.7418 - val_loss: 0.4994
Epoch 3/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 83s 46ms/step - accuracy: 0.7438 - loss: 0.4976 - val_accuracy: 0.7417 - val_loss: 0.4963
Epoch 4/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.7415 - loss: 0.4977 - val_accuracy: 0.7423 - val_loss: 0.4929
Epoch 5/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 73s 45ms/step - accuracy: 0.7426 - loss: 0.4945 - val_accuracy: 0.7419 - val_loss: 0.4939
Epoch 6/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 76s 47ms/step - accuracy: 0.7432 - loss: 0.4951 - val_accuracy: 0.7428 - val_loss: 0.4930
Epoch 7/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 71s 44ms/step - accuracy: 0.7446 - loss: 0.4931 - val_accuracy: 0.7419 - val_loss: 0.4926
Epoch 8/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.7428 -

# Predict & Submission

In [30]:
y_test = model.predict(X_test_encoded) #다른 모델 예측하려면 cat_model을 다른 모델명으로 바꿔주면 됨.

2815/2815 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step


In [31]:
sample_submission = pd.read_csv('/content/drive/My Drive/Aimers6/sample_submission.csv')
sample_submission['probability'] = y_test

sample_submission.to_csv('/content/drive/My Drive/Aimers6/relu_submission.csv', index=False)